<a href="https://colab.research.google.com/github/lkarjun/fastai-workouts/blob/master/Lesson_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packages

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 8.8MB/s 
     |████████████████████████████████| 1.2MB 32.1MB/s 
     |████████████████████████████████| 194kB 50.0MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
     |████████████████████████████████| 61kB 10.3MB/s 
Mounted at /content/gdrive


In [2]:
from fastai.text.all import *

# Data Munging

In [ ]:
dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')

In [3]:
path = untar_data(URLs.IMDB)

dls = DataBlock(
    blocks = (TextBlock.from_folder(path), CategoryBlock()),
    get_y = parent_label,
    get_items = partial(get_text_files, folders = ['train', 'test']),
    splitter = GrandparentSplitter(valid_name = 'test')
).dataloaders(path)

## Transforms

In [6]:
files = get_text_files(path, folders=['train', 'test'])
txts = L(o.open().read() for o in files[:2000])

In [7]:
tok = Tokenizer.from_folder(path)
tok.setup(txts)
toks = txts.map(tok)
toks[0]

(#278) ['xxbos','xxmaj','despite','having','a','very','pretty','leading','lady','('...]

In [8]:
num = Numericalize()
num.setup(toks)
nums = toks.map(num)
nums[0][:5]

TensorText([  2,   8, 529, 269,  12])

In [9]:
nums_dec = num.decode(nums[0][:10])
nums_dec

(#10) ['xxbos','xxmaj','despite','having','a','very','pretty','leading','lady','(']

In [10]:
tok.decode(toks[0][:10])

'xxbos xxmaj despite having a very pretty leading lady ('

## Writing Your Own Transform

In [12]:
def f(x: int): return x+1

tfm = Transform(f)
tfm(2), tfm(2.0)

(3, 2.0)

In [ ]:
Transform??

In [14]:
@Transform
def f(x: int): return x+1 # It will only work when the input == int type

f(2), f(2.0)

(3, 2.0)

In [16]:
class NormalizeMean(Transform):
  def setup(self, items): self.mean = sum(items)/len(items)
  def encodes(self, x): return x - self.mean
  def decodes(self, x): return x + self.mean

In [19]:
tfm = NormalizeMean()
tfm.setup([1,2,3,4,5])

In [20]:
print("Mean is: ", tfm.mean)
enco = tfm(2)
print("Encode of 2 is : ", enco)
print("Decode of enco is: ", tfm.decode(enco))

Mean is:  3.0
Encode of 2 is :  -1.0
Decode of enco is:  2.0
